In [ ]:
#KMEANS_SCAN_ANALYSIS

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from mypackage.MyMethods import *

import pandas as pd
import numpy as np
from plotnine import * 
from ots_eval.stability_evaluation.close import CLOSE

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

from matplotlib import pyplot as plt

%matplotlib inline

In [ ]:
d1 = pd.read_csv("datasets/generated_1d_data.csv")

z = StandardScaler()
d1[["time", "feature1"]] = z.fit_transform(d1[["time", "feature1"]])

ggplot(d1, aes(x = "time", y = "feature1")) + geom_point() + theme_minimal()

In [ ]:
#Nicht Vorhandene Werte von Zeitreihen simulieren
sample_df = d1.sample(frac =.8)

In [ ]:
#defekte (None) Werte von Zeitreihe simulieren
dfupdate = sample_df.sample(frac = .2).index
sample_df.loc[dfupdate, "feature1"] = np.NaN

In [ ]:
#alle Rows mit NaN Werten droppen
sample_df.dropna(subset=["object_id", "time", "feature1"], inplace=True)

In [ ]:
test_db1 = KMeans(23).fit(sample_df[["time", "feature1"]])

labsList = ["Cluster " + str(i) for i in range(0, len(set(test_db1.labels_)))]

sample_df["assignments"] = test_db1.labels_

(ggplot(sample_df, aes(x = "time", y = "feature1", color = "factor(assignments)")) + geom_point() + theme_minimal() +
    scale_color_discrete(name = "Cluster Assignment",
                        labels = labsList) +
theme(panel_grid_major = element_blank()) +
labs(title = "DBSCAN with eps = ?, min_samples = ? "))

In [ ]:
df_clustered = sample_df.loc[(sample_df.assignments >= 0)]

silhouette_score(df_clustered[["time", "feature1"]], df_clustered[["assignments"]])

In [ ]:
rater = CLOSE(sample_df[['object_id', 'time', 'assignments', 'feature1']], 'mae', 2, jaccard=True, weighting=True, exploitation_term=True)
print(rater.rate_clustering())
rater = CLOSE(sample_df[['object_id', 'time', 'assignments', 'feature1']], 'exploit', 2, jaccard=True, weighting=True)
print(rater.rate_time_clustering())

In [ ]:
#Spalten die gescaled wurden wieder zurückscalen
d1[["time", "feature1"]] = z.inverse_transform(d1[["time", "feature1"]])
sample_df[["time", "feature1"]] = z.inverse_transform(sample_df[["time", "feature1"]])

In [ ]:
missing_rows = get_missing_rows(sample_df, "object_id", "time")
for ids, time in missing_rows:
    print(d1[(d1["object_id"] == ids) & (d1["time"] == time)].values)

In [ ]:
most_frequent_cluster_member(sample_df, "object_id", "time", "feature1", "assignments")

In [ ]:
pre_and_post_clustering_analysis(sample_df, "object_id", "time", "feature1", "assignments")

In [ ]:
mean_of_all(sample_df, "object_id", "time", "feature1", "assignments")

In [ ]:
median_of_all(sample_df, "object_id", "time", "feature1", "assignments")

In [ ]:
median_of_all_test(sample_df,"object_id",  "time", "feature1", "assignments")

In [ ]:
sample_df